In [35]:
#Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import time
from numba import jit, prange

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# read data
data=pd.read_csv('/content/drive/My Drive/data_standardize.csv')
df_Standardize=pd.DataFrame(data)
df_Standardize

,Unnamed: 0,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,0,-0.745369,-0.248785,-0.465787,-0.397088,-0.365854,-0.476844,-0.805116,-0.677987,-0.706172,-0.675708,-0.482201,-0.520766,-0.965868,-0.549205,-0.324378,-0.525663,0.360754
1,1,0.822978,0.134866,-0.516362,-0.397088,-0.479339,2.682990,-1.220469,-0.677987,-0.915974,0.576272,0.114210,-0.603824,0.705048,0.877253,0.123457,0.234367,0.360754
2,2,0.472101,0.518517,-0.106471,0.140262,-0.479339,-0.476844,1.271643,2.681612,-0.915974,-0.675708,-0.482201,-0.105476,0.844291,-0.395538,-0.090206,-0.525663,0.360754
3,3,0.061121,-1.016087,0.278323,0.644711,-0.479339,-0.375919,-1.012794,-0.398022,-0.915974,-0.258383,-0.333098,-0.562295,0.844291,-0.622992,-0.391354,-0.525663,0.360754
4,4,-0.360018,0.518517,-0.507880,-0.385968,-0.479339,-0.476844,-1.012794,-0.398022,-0.915974,-0.675708,-0.482201,-0.562295,-0.910171,-0.374964,-0.273834,-0.525663,0.360754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8926,8926,-0.751524,0.518517,-0.362027,-0.397088,-0.133031,-0.476844,1.271643,-0.677987,1.182060,-0.675708,-0.482201,-0.354650,-0.965868,-0.503941,-0.367885,1.184407,-4.124481
8927,8927,-0.756142,0.518517,-0.357319,-0.397088,-0.122468,-0.476844,1.271643,-0.677987,1.182060,-0.675708,-0.482201,-0.354650,-0.965868,-0.522125,-0.258918,-0.525663,-4.124481
8928,8928,-0.754051,-0.184845,-0.439810,-0.397088,-0.307565,-0.476844,0.856290,-0.677987,0.762454,-0.675708,-0.482201,-0.396179,-0.965868,-0.593276,-0.351786,0.329372,-4.124481
8929,8929,-0.758982,-0.184845,-0.516362,-0.397088,-0.479339,-0.458914,-1.220469,-0.677987,-0.915974,0.158947,-0.183995,-0.603824,-1.105111,-0.603778,-0.364587,0.329372,-4.124481


In [47]:
K = 3
df_Standardize=df_Standardize.to_numpy()

Using numba to optimize sequential KMeans code

In [48]:
@jit()
# initial centers
def kmeans_init_centers(df_Standardize, K):
    centers = np.empty((K, df_Standardize.shape[1]))
    for i in range(K):
        index = np.random.choice(df_Standardize.shape[0])
        centers[i, :] = df_Standardize[index, :]
    return centers

<ipython-input-48-4481fbd6117e>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [49]:
@jit()
# calculate distance between data points using Euclid distance
def kmeans_cal_distance(point_1, point_2):
    sum_point = 0.0
    for i in range(point_1.shape[0]):
        sum_point += (point_1[i] - point_2[i]) ** 2
    return np.sqrt(sum_point)

<ipython-input-49-64de4ff6742b>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [50]:
@jit()
# assign label for each point corresponding to each cluster that point related
def kmeans_assign_labels(df_Standardize, centers):
    assigned_labels = np.empty(df_Standardize.shape[0], dtype=np.int32)
    for i in range(df_Standardize.shape[0]):
        distance = np.empty(centers.shape[0])
        for j in range(centers.shape[0]):
            distance[j] = kmeans_cal_distance(centers[j, :], df_Standardize[i, :])
        assigned_labels[i] = np.argmin(distance)
    return assigned_labels

<ipython-input-50-65fafa1095a3>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [51]:
# update new centroids
@jit()
def kmeans_update_centroids(df_Standardize, labels, K):
    centers = np.zeros((K, df_Standardize.shape[1]))
    counts = np.zeros(K, dtype=np.int32)
    for i in range(df_Standardize.shape[0]):
        label = labels[i]
        centers[label, :] += df_Standardize[i, :]
        counts[label] += 1

    for k in range(K):
        if counts[k] > 0:
            centers[k, :] /= counts[k]

    return centers

<ipython-input-51-224023e5bab3>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [52]:
# stop condition
@jit()
def has_converged(centers, new_centers):
    # return True if two sets of centers are the same
    for i in range(len(centers)):
        for j in range(len(centers[i])):
            if centers[i][j] != new_centers[i][j]:
                return False
    return True

<ipython-input-52-c2374751bcd1>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [53]:
@jit()
# main kmeans function
def kmeans(df_Standardize, K):
    centers = kmeans_init_centers(df_Standardize, K)
    it = 0
    while True:
        assigned_labels = kmeans_assign_labels(df_Standardize, centers)
        new_centers = kmeans_update_centroids(df_Standardize, assigned_labels, K)
        if has_converged(centers, new_centers):
            break
        centers = new_centers
        it += 1
    return centers, assigned_labels, it

<ipython-input-53-2a5a6f0876ea>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [54]:
start = time.time()
(centers, labels, it) = kmeans(df_Standardize, K)
end = time.time()
print(f'Processing time: {end - start} s')
print('Centers found by k-means algorithm:')
print(centers[-1])
print('='*60)

Processing time: 2.3774335384368896 s
Centers found by k-means algorithm:
[ 4.46700000e+03  3.81094862e-02  5.02189895e-02  5.83274799e-04
  5.36927127e-03 -7.84065808e-03  6.31988406e-03 -4.67442461e-02
  4.47162453e-02 -6.93487645e-02  1.87956259e-02  1.88486182e-02
 -3.75253784e-03  7.48275532e-02  1.65785762e-02  4.62410403e-03
 -2.75550847e-02  7.75076913e-02]


Using numba to covert sequential KMeans code to machine code and run parallel on CPU

In [55]:
@jit()
# initial centers
def kmeans_init_centers(df_Standardize, K):
    centers = np.empty((K, df_Standardize.shape[1]))
    for i in range(K):
        index = np.random.choice(df_Standardize.shape[0])
        centers[i, :] = df_Standardize[index, :]
    return centers

<ipython-input-55-4481fbd6117e>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [56]:
@jit(parallel=True)
# calculate distance between data points using Euclid distance
def kmeans_cal_distance(point_1, point_2):
    sum_point = 0.0
    for i in prange(point_1.shape[0]):
        sum_point += (point_1[i] - point_2[i]) ** 2
    return np.sqrt(sum_point)

<ipython-input-56-6db6717e833a>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)


In [57]:
@jit(parallel=True)
# assign label for each point corresponding to each cluster that point related
def kmeans_assign_labels(df_Standardize, centers):
    assigned_labels = np.empty(df_Standardize.shape[0], dtype=np.int32)
    for i in prange(df_Standardize.shape[0]):
        distance = np.empty(centers.shape[0])
        for j in range(centers.shape[0]):
            distance[j] = kmeans_cal_distance(centers[j, :], df_Standardize[i, :])
        assigned_labels[i] = np.argmin(distance)
    return assigned_labels

<ipython-input-57-7b3b6e2f774f>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)


In [58]:
# update new centroids
@jit(parallel=True)
def kmeans_update_centroids(df_Standardize, labels, K):
    centers = np.zeros((K, df_Standardize.shape[1]))
    counts = np.zeros(K, dtype=np.int32)
    for i in prange(df_Standardize.shape[0]):
        label = labels[i]
        centers[label, :] += df_Standardize[i, :]
        counts[label] += 1

    for k in prange(K):
        if counts[k] > 0:
            centers[k, :] /= counts[k]

    return centers

<ipython-input-58-8a0ec14d7fd5>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(parallel=True)


In [59]:
@jit()
# stop condition
def has_converged(centers, new_centers):
    for i in range(centers.shape[0]):
        for j in range(centers.shape[1]):
            if centers[i, j] != new_centers[i, j]:
                return False
    return True

<ipython-input-59-d7e4c81a98a7>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [60]:
@jit()
# main kmeans function
def kmeans(df_Standardize, K):
    centers = kmeans_init_centers(df_Standardize, K)
    it = 0
    while True:
        assigned_labels = kmeans_assign_labels(df_Standardize, centers)
        new_centers = kmeans_update_centroids(df_Standardize, assigned_labels, K)
        if has_converged(centers, new_centers):
            break
        centers = new_centers
        it += 1
    return centers, assigned_labels, it

<ipython-input-60-2a5a6f0876ea>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit()


In [61]:
start = time.time()
(centers, labels, it) = kmeans(df_Standardize, K)
end = time.time()
print(f'Processing time: {end - start} s')
print('Centers found by k-means algorithm:')
print(centers[-1])
print('='*60)

Processing time: 8.225087404251099 s
Centers found by k-means algorithm:
[ 4.46700000e+03  3.81094862e-02  5.02189895e-02  5.83274799e-04
  5.36927127e-03 -7.84065808e-03  6.31988406e-03 -4.67442461e-02
  4.47162453e-02 -6.93487645e-02  1.87956259e-02  1.88486182e-02
 -3.75253784e-03  7.48275532e-02  1.65785762e-02  4.62410403e-03
 -2.75550847e-02  7.75076913e-02]
